<a href="https://colab.research.google.com/github/joeldmott/chicago_auto_accidents_project/blob/main/data_engineering_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determining the Primary Causes of Traffic Accidents in Chicago: Data Engineering Notebook

## summary

This notebook serves as a precursor to the [project notebook](https://github.com/joeldmott/chicago_auto_accidents_project/blob/main/project_notebook.ipynb) for this project, where I trim the dimensions of a 146-row dataset down to 15 using domain knowledge and keeping the project's overall goal in mind.

This project's raw dataset originates from the [City of Chicago's website](https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if/about_data), where it is updated daily. I downloaded the data on May 1st, 2024 and [uploaded that snapshot to Kaggle](https://www.kaggle.com/datasets/joelmott/chicago-traffic-crashes-may-2024).

This dataset consists of three seperate csv files: one for general crash information, one for the people involved in each crash, and one for each vehicle. When merged, the resulting dataset contains over 146 columns and 3.8 million records tied to around 837,000 different traffic crashes.

Since the purpose is to take a broader look at traffic incidents in Chicago as opposed to just those crashes that resulted in injuries or fatalities, I eliminate a number of columns that pertain to features that are more niche or don't relate as much to crashes in general. I also eliminate columns that ncessarily interrelate or are redunandant.

The resulting trimmed, single CSV file was then uploaded to my Kaggle page (referenced above) and is used in the [project notebook](https://github.com/joeldmott/chicago_auto_accidents_project/blob/main/project_notebook.ipynb) for modeling.

## downloading & importing the data

Below, we start by importing the packages we need to read the dataset in from Kaggle, then downloading each CSV file before beginning with some EDA.

In [ ]:
import json
import os
from pathlib import Path
from google.colab import userdata

# api key for importing Kaggle and downloading the datasets
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
# api key for the json object below
api_key = userdata.get('API_KEY')

# uses pathlib Path
kaggle_path = Path('/root/.kaggle')
os.makedirs(kaggle_path, exist_ok=True)

# opens file and dumps python dict to json object
with open (kaggle_path/'kaggle.json', 'w') as handl:
    json.dump(api_key,handl)

os.chmod(kaggle_path/'kaggle.json', 600)

In [ ]:
import kaggle
! kaggle datasets download joelmott/chicago-traffic-crashes-may-2024/ -f CPD_traffic_crashes-crashes-20240501.csv
! kaggle datasets download joelmott/chicago-traffic-crashes-may-2024/ -f CPD_traffic_crashes-people-20240501.csv
! kaggle datasets download joelmott/chicago-traffic-crashes-may-2024/ -f CPD_traffic_crashes-vehicles-20240501.csv

Dataset URL: https://www.kaggle.com/datasets/joelmott/chicago-traffic-crashes-may-2024/versions/
License(s): CC0-1.0
CPD_traffic_crashes-crashes-20240501.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/joelmott/chicago-traffic-crashes-may-2024/versions/
License(s): CC0-1.0
CPD_traffic_crashes-people-20240501.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/joelmott/chicago-traffic-crashes-may-2024/versions/
License(s): CC0-1.0
CPD_traffic_crashes-vehicles-20240501.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#unzipping files
!unzip /content/CPD_traffic_crashes-crashes-20240501.csv.zip
!unzip /content/CPD_traffic_crashes-people-20240501.csv.zip
!unzip /content/CPD_traffic_crashes-vehicles-20240501.csv.zip

Archive:  /content/CPD_traffic_crashes-crashes-20240501.csv.zip
replace CPD_traffic_crashes-crashes-20240501.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/CPD_traffic_crashes-people-20240501.csv.zip
replace CPD_traffic_crashes-people-20240501.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/CPD_traffic_crashes-vehicles-20240501.csv.zip
replace CPD_traffic_crashes-vehicles-20240501.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#reading in the first file (on general crash info)
import pandas as pd
crashes_df = pd.read_csv('/content/CPD_traffic_crashes-crashes-20240501.csv')
crashes_df.head()

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,NaN,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,...,1.0,0.0,1.0,0.0,12,6,8,NaN,NaN,NaN
1,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,NaN,07/29/2023 02:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,1.0,0.0,14,7,7,41.854120,-87.665902,POINT (-87.665902342962 41.854120262952)
2,61fcb8c1eb522a6469b460e2134df3d15f82e81fd93e9c...,NaN,08/18/2023 05:58:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDALCYCLIST,NOT DIVIDED,...,1.0,0.0,1.0,0.0,17,6,8,41.942976,-87.761883,POINT (-87.761883496974 41.942975745006)
3,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,NaN,11/26/2019 08:38:00 AM,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDESTRIAN,ONE-WAY,...,0.0,0.0,1.0,0.0,8,3,11,NaN,NaN,NaN
4,a1d5f0ea90897745365a4cbb06cc60329a120d89753fac...,NaN,08/18/2023 10:45:00 AM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,OTHER,...,0.0,0.0,1.0,0.0,10,6,8,NaN,NaN,NaN


## EDA

Below, I work to understand the nature of each column in the dataset in order to make sure important ones for general accident causes are kept and unnecessary ones are eliminated.

In [ ]:
crashes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828511 entries, 0 to 828510
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                828511 non-null  object 
 1   CRASH_DATE_EST_I               61842 non-null   object 
 2   CRASH_DATE                     828511 non-null  object 
 3   POSTED_SPEED_LIMIT             828511 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         828511 non-null  object 
 5   DEVICE_CONDITION               828511 non-null  object 
 6   WEATHER_CONDITION              828511 non-null  object 
 7   LIGHTING_CONDITION             828511 non-null  object 
 8   FIRST_CRASH_TYPE               828511 non-null  object 
 9   TRAFFICWAY_TYPE                828511 non-null  object 
 10  LANE_CNT                       199011 non-null  float64
 11  ALIGNMENT                      828511 non-null  object 
 12  ROADWAY_SURFACE_COND          

Keeping the project's prompt in mind, it looks like 'PRIM_CONTRIBUTORY_CAUSE' is our main target column.

As far as feature engineering goes, there are several different already-ohe'd categorical injury columns, maybe you add up all the ones with fatalities and bad injuries or something.

Next would be getting a feel for the column values where the data description isn't already obvious.

In [ ]:
crashes_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 322717
FAILING TO YIELD RIGHT-OF-WAY                                                        91120
FOLLOWING TOO CLOSELY                                                                80452
NOT APPLICABLE                                                                       43885
IMPROPER OVERTAKING/PASSING                                                          40874
FAILING TO REDUCE SPEED TO AVOID CRASH                                               34961
IMPROPER BACKING                                                                     32492
IMPROPER LANE USAGE                                                                  29635
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  27817
IMPROPER TURNING/NO SIGNAL                                                           27539
DISREGARDING TRAFFIC SIGNALS                                      

In [ ]:
len(crashes_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts())

40

40 classes?!? Some of them occured fewer than 100 times! I guess I'd have to drop the 'UNABLE TO DETERMINE' and 'NOT APPLICABLE' rows. Then, I think I'd need to group the remaining 38 classes together somehow.

I'll work on that off-and-on while I continue to check out the remaining features in this crashes DataFrame. Let's start with 'TRAFFIC_CONTROL_DEVICE':

In [ ]:
crashes_df['TRAFFIC_CONTROL_DEVICE'].value_counts()

TRAFFIC_CONTROL_DEVICE
NO CONTROLS                 470741
TRAFFIC SIGNAL              229645
STOP SIGN/FLASHER            82248
UNKNOWN                      33447
OTHER                         5623
LANE USE MARKING              1226
YIELD                         1218
OTHER REG. SIGN                932
OTHER WARNING SIGN             682
RAILROAD CROSSING GATE         536
PEDESTRIAN CROSSING SIGN       532
FLASHING CONTROL SIGNAL        334
DELINEATORS                    331
SCHOOL ZONE                    315
POLICE/FLAGMAN                 279
OTHER RAILROAD CROSSING        182
RR CROSSING SIGN               164
NO PASSING                      51
BICYCLE CROSSING SIGN           25
Name: count, dtype: int64

This feature seems to be almost enitrely made up of the classes 'no controls', 'traffic signal', 'stop sign/flasher', or 'unknown.' The rest are quite rare, relatively speaking. I think this could be a helpful column, but the distribution is really skewed, so in the interest of trimming features that may add more noise to our model when model interpretability is a priority, let's drop this one.

Let's move on to the 'DEVICE_CONDITION' feature:

In [ ]:
crashes_df['DEVICE_CONDITION'].value_counts()

DEVICE_CONDITION
NO CONTROLS                 476188
FUNCTIONING PROPERLY        283564
UNKNOWN                      55753
OTHER                         6336
FUNCTIONING IMPROPERLY        3890
NOT FUNCTIONING               2397
WORN REFLECTIVE MATERIAL       287
MISSING                         96
Name: count, dtype: int64

I had to look it up, but 'NO CONTROLS' refers to a traffic control device that doesn't change like a light would, but rather is a permanent fixture like a sign or road markings. I guess it's not surprising that the vast majority are in this category and then another huge chunk are at intersections where the light was working properly. Note a decent amount of 'UNKNOWN' entries here. With only 3,890 functioning improperly compared to 759,752 either not having controls or working normally, I don't think this feature will be very helpful, so let's drop it.

Next to check out is 'FIRST_CRASH_TYPE':

In [ ]:
crashes_df['FIRST_CRASH_TYPE'].value_counts()

FIRST_CRASH_TYPE
PARKED MOTOR VEHICLE            192428
REAR END                        184855
SIDESWIPE SAME DIRECTION        126534
TURNING                         118537
ANGLE                            90173
FIXED OBJECT                     38930
PEDESTRIAN                       19330
PEDALCYCLIST                     12448
SIDESWIPE OPPOSITE DIRECTION     11666
OTHER OBJECT                      8216
REAR TO FRONT                     8094
HEAD ON                           7061
REAR TO SIDE                      4807
OTHER NONCOLLISION                2606
REAR TO REAR                      1676
ANIMAL                             596
OVERTURNED                         508
TRAIN                               46
Name: count, dtype: int64

I can't help but wonder whether this is a good *predictor* for an accident's primary cause when it describes the instance of the accident, not what happens beforehand. It turns out that, later on, we'll find a better column for this in the vehicles DataFrame ('MANEUVER') that will describe the action that led to the accident, so let's drop this column in favor of that one.

The following 'TRAFFICWAY_TYPE' column looks useful as well, but it has so many classes that, along with the other useful features we'll eventually find, it'll wind up adding a lot of noise to our data with the sparsity that will result once it's one-hot encoded.

In [ ]:
crashes_df['TRAFFICWAY_TYPE'].value_counts()

TRAFFICWAY_TYPE
NOT DIVIDED                        359452
DIVIDED - W/MEDIAN (NOT RAISED)    132085
ONE-WAY                            105558
PARKING LOT                         56368
FOUR WAY                            53091
DIVIDED - W/MEDIAN BARRIER          47287
OTHER                               22646
ALLEY                               13690
T-INTERSECTION                      10660
UNKNOWN                              9656
CENTER TURN LANE                     5978
DRIVEWAY                             2691
RAMP                                 2595
UNKNOWN INTERSECTION TYPE            2455
Y-INTERSECTION                       1164
FIVE POINT, OR MORE                  1161
TRAFFIC ROUTE                         948
NOT REPORTED                          592
ROUNDABOUT                            262
L-INTERSECTION                        172
Name: count, dtype: int64

'ALIGNMENT' has a distribution issue with the top class making up almost 98% of this feature, so it won't be too helpful.

In [ ]:
crashes_df['ALIGNMENT'].value_counts()

ALIGNMENT
STRAIGHT AND LEVEL       808482
STRAIGHT ON GRADE         10279
CURVE, LEVEL               5971
STRAIGHT ON HILLCREST      2169
CURVE ON GRADE             1243
CURVE ON HILLCREST          367
Name: count, dtype: int64

'CRASH_TYPE' would make for a helpful target column if we were interested in less serious versus more serious accidents, but we're looking into the cause of *all* accidents and this column ultimately describes the aftermath, not the cause, so we'll drop for this particular use case.

In [ ]:
crashes_df['CRASH_TYPE'].value_counts()

CRASH_TYPE
NO INJURY / DRIVE AWAY              607050
INJURY AND / OR TOW DUE TO CRASH    221461
Name: count, dtype: int64

The following column covers whether a right-of-way violation occured, but that's in our target variable, so we should eliminate this one.

In [ ]:
crashes_df['NOT_RIGHT_OF_WAY_I'].value_counts()

NOT_RIGHT_OF_WAY_I
Y    34674
N     3493
Name: count, dtype: int64

The 'DAMAGE' column refers to the aftermath, not the cause, so this can be dropped.

In [ ]:
crashes_df['DAMAGE'].value_counts()

DAMAGE
OVER $1,500      515387
$501 - $1,500    218754
$500 OR LESS      94370
Name: count, dtype: int64

These 'BEAT_OF_OCCURANCE' classes are categorical even if they're listed here as numbers. Subsequently, they would need to be one-hot encoded, and there are just far too many of them for that, so we'll have to drop this column as well.

In [ ]:
crashes_df['BEAT_OF_OCCURRENCE'].value_counts()

BEAT_OF_OCCURRENCE
1834.0    10067
114.0      8490
813.0      8332
815.0      7899
1831.0     7678
          ...  
1653.0      430
1655.0      260
1652.0      200
1650.0       69
6100.0        4
Name: count, Length: 276, dtype: int64

There are probably a ton of unique values in the latitude & longitude columns, but let's make sure they at least make sense.

In [ ]:
crashes_df['LATITUDE'].value_counts()

LATITUDE
41.976201    1319
41.900959     772
41.791420     586
41.751461     560
41.722257     450
             ... 
41.917967       1
41.731148       1
41.976312       1
41.866965       1
41.923913       1
Name: count, Length: 302547, dtype: int64

302,547 is a lot of different values. It seems like they're all clustered around 41 degrees north, let's see if there are any outliers as placeholders for NaN's like '99' or something.

In [ ]:
crashes_df[crashes_df['LATITUDE'] > 43]

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION


In [ ]:
crashes_df[crashes_df['LATITUDE'] < 40]

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
746,030f51ced858d85f09a69770f563cc74f8d3469eb3164b...,NaN,01/06/2019 01:28:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,...,0.0,0.0,2.0,0.0,13,1,1,0.0,0.0,POINT (0 0)
892,0837e4a0611e47baed64af145b8256b1b63f877b5d0268...,NaN,01/09/2018 01:10:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,...,0.0,0.0,2.0,0.0,13,3,1,0.0,0.0,POINT (0 0)
1410,139ae18cce07ef02dff876ef818fdc5357f27cdc0dd06c...,Y,11/13/2022 11:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,...,0.0,0.0,2.0,0.0,23,1,11,0.0,0.0,POINT (0 0)
1532,15a3ce621f48c980c801e9c7f8b6da22fc29c95a26f9c6...,NaN,02/08/2019 04:14:00 PM,30,TRAFFIC SIGNAL,UNKNOWN,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,...,0.0,1.0,1.0,0.0,16,6,2,0.0,0.0,POINT (0 0)
1756,1926313099fc22b23e2a647e3af4d83eb8f41b7cdb00a0...,NaN,05/11/2018 05:55:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAWN,TURNING,OTHER,...,1.0,0.0,3.0,0.0,5,6,5,0.0,0.0,POINT (0 0)
1868,1c22fc3438a78f813ff26f25f475ec8c90d85c6f698a41...,NaN,11/08/2016 03:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,...,0.0,0.0,2.0,0.0,15,3,11,0.0,0.0,POINT (0 0)
1918,1f666a133a36453c07641c3fa1a73f69cdf01e6b03e757...,NaN,01/02/2019 11:05:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,DAYLIGHT,REAR END,NOT DIVIDED,...,0.0,0.0,2.0,0.0,11,4,1,0.0,0.0,POINT (0 0)
2150,217caa6775d3cb36ae884bb5b80a7629d05108371fd7d6...,NaN,07/31/2017 06:45:00 PM,30,TRAFFIC SIGNAL,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,...,0.0,1.0,1.0,0.0,18,2,7,0.0,0.0,POINT (0 0)
2403,2588f9b181d03bfe3de43c5e38f37c4f8665092334654c...,NaN,06/19/2019 09:15:00 AM,30,STOP SIGN/FLASHER,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,...,0.0,0.0,2.0,0.0,9,4,6,0.0,0.0,POINT (0 0)
2527,29024d2a4632df975ac68682c77c3340df99d672d4f4f3...,NaN,10/28/2017 08:00:00 PM,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,3.0,0.0,20,7,10,0.0,0.0,POINT (0 0)


There are a handful of records with "0.0" for latitude & longitude, which is essentially a NaN, so let's drop those.

In [ ]:
crashes_df = crashes_df[crashes_df['LATITUDE'] != 0]
crashes_df[crashes_df['LATITUDE'] < 40]

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION


Now that I have a sense of what all these columns mean, it's time to consider which enitre features should be eliminated to help reduce dimensionality. This is especially important since so many of these columns will have to be one-hot encoded.

Since the goal of this project is to predict the *primary cause* of an accident, we might consider dropping those columns that inherently have nothing to do with a potential cause (such as 'DATE_POLICE_NOTIFIED', 'STREET NO', 'STREET NAME', 'PHOTOS TAKEN', 'STATEMENTS TAKEN', 'REPORT_TYPE') and/or only pertain to the effect (all the damage, injury, and fatality columns). We can also drop the 'SEC_CONTRIBUTORY_CAUSE' or we'll probably have a lot of multicollinearity issues.

There are a few redundant columns for our specific use case. 'LOCATION' just puts the latitude & longitude columns together. Since those two columns will provide location data *and* 'BEAT_OF_OCCURRENCE' has 276 categorical values, let's eliminate the beat column as well. Also, 'CRASH_DATE_EST_I' and 'CRASH_DATE' are broken down later via 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', and 'CRASH_MONTH'.

Finally, some columns are woefully empty; 'LANE_CNT', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DOORING_I', and all the work zone columns only have somewhere between 68.7-99% empty, which is grounds for dropping.

I'll keep 'CRASH_RECORD_ID' for now so I can join the 'people' & 'vehicles' tables later.

In [ ]:
crashes_columns_to_drop = ['CRASH_DATE', 'CRASH_DATE_EST_I', 'DEVICE_CONDITION', 'FIRST_CRASH_TYPE', 'CRASH_TYPE',
                           'REPORT_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED', 'SEC_CONTRIBUTORY_CAUSE',
                           'TRAFFIC_CONTROL_DEVICE', 'STREET_NO', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I',
                           'STATEMENTS_TAKEN_I', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
                           'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
                           'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'LOCATION', 'LANE_CNT', 'ALIGNMENT',
                           'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DOORING_I',
                           'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'TRAFFICWAY_TYPE']

In [ ]:
crashes_df_refined = crashes_df.drop(crashes_columns_to_drop, axis=1)
crashes_df_refined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 828462 entries, 0 to 828510
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CRASH_RECORD_ID          828462 non-null  object 
 1   POSTED_SPEED_LIMIT       828462 non-null  int64  
 2   WEATHER_CONDITION        828462 non-null  object 
 3   LIGHTING_CONDITION       828462 non-null  object 
 4   ROADWAY_SURFACE_COND     828462 non-null  object 
 5   ROAD_DEFECT              828462 non-null  object 
 6   PRIM_CONTRIBUTORY_CAUSE  828462 non-null  object 
 7   STREET_DIRECTION         828458 non-null  object 
 8   NUM_UNITS                828462 non-null  int64  
 9   CRASH_HOUR               828462 non-null  int64  
 10  CRASH_DAY_OF_WEEK        828462 non-null  int64  
 11  CRASH_MONTH              828462 non-null  int64  
 12  LATITUDE                 822720 non-null  float64
 13  LONGITUDE                822720 non-null  float64
dtypes: float6

Now that we've trimmed down the 'crashes' data, let's do the same with the second table on the people involved.

## people data

In [ ]:
people_df = pd.read_csv('/content/CPD_traffic_crashes-people-20240501.csv')
people_df.head(3)

<ipython-input-23-26241325ca0e>:1: DtypeWarning: Columns (19,23,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  people_df = pd.read_csv('/content/CPD_traffic_crashes-people-20240501.csv')


,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,M,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,M,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [ ]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1818705 entries, 0 to 1818704
Data columns (total 29 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   PERSON_ID              object 
 1   PERSON_TYPE            object 
 2   CRASH_RECORD_ID        object 
 3   VEHICLE_ID             float64
 4   CRASH_DATE             object 
 5   SEAT_NO                float64
 6   CITY                   object 
 7   STATE                  object 
 8   ZIPCODE                object 
 9   SEX                    object 
 10  AGE                    float64
 11  DRIVERS_LICENSE_STATE  object 
 12  DRIVERS_LICENSE_CLASS  object 
 13  SAFETY_EQUIPMENT       object 
 14  AIRBAG_DEPLOYED        object 
 15  EJECTION               object 
 16  INJURY_CLASSIFICATION  object 
 17  HOSPITAL               object 
 18  EMS_AGENCY             object 
 19  EMS_RUN_NO             object 
 20  DRIVER_ACTION          object 
 21  DRIVER_VISION          object 
 22  PHYSICAL_CONDITION

As with the crashes set, our first task is to understand and then reduce the columns we'll use. Specifically, we'll determine & eliminate those that don't necessarily pertain to the the *cause* of an accident, but rather the aftermath. In order to decrease dimensionality, we'll also drop categorical columns that have high cardinality and aren't likely to contribute much, like zipcodes or the state from which the drivers license was issued.

In [ ]:
people_df['PERSON_TYPE'].value_counts()

PERSON_TYPE
DRIVER                 1415231
PASSENGER               367648
PEDESTRIAN               21244
BICYCLE                  12794
NON-MOTOR VEHICLE         1482
NON-CONTACT VEHICLE        306
Name: count, dtype: int64

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (people_df['DRIVER_ACTION'].value_counts())

DRIVER_ACTION
NONE                                 517295
UNKNOWN                              365078
FAILED TO YIELD                      131914
OTHER                                129661
FOLLOWED TOO CLOSELY                  85675
IMPROPER BACKING                      42948
IMPROPER TURN                         37980
IMPROPER LANE CHANGE                  37173
IMPROPER PASSING                      32369
DISREGARDED CONTROL DEVICES           25564
TOO FAST FOR CONDITIONS               21734
WRONG WAY/SIDE                         5813
IMPROPER PARKING                       5329
OVERCORRECTED                          2710
EVADING POLICE VEHICLE                 2304
CELL PHONE USE OTHER THAN TEXTING      2126
EMERGENCY VEHICLE ON CALL              1328
TEXTING                                 569
STOPPED SCHOOL BUS                      174
LICENSE RESTRICTIONS                     61
Name: count, dtype: int64


In [ ]:
people_df['DRIVER_VISION'].value_counts()

DRIVER_VISION
NOT OBSCURED              731001
UNKNOWN                   681356
OTHER                      14386
MOVING VEHICLES             8198
PARKED VEHICLES             5044
WINDSHIELD (WATER/ICE)      3987
BLINDED - SUNLIGHT          1716
TREES, PLANTS                578
BUILDINGS                    516
BLINDED - HEADLIGHTS         155
BLOWING MATERIALS            102
HILLCREST                     96
EMBANKMENT                    83
SIGNBOARD                     35
Name: count, dtype: int64

Safety equipment almost certainly has an impact on injuries & fatalities, but if we're prioritizing an effort to minimize dimensions, then weighing whether they have an impact on the *cause* probably means we can drop this column.

'DRIVER_ACTION' can be dropped since it's pretty redundant when it comes to our target. 'DRIVER_VISION' categorizes whether and how the driver's vision was impeded, but its classes are almost enitrely either 'unobscured' or 'unknown', so I don't think it will helpful here. The same goes for 'PHYSICAL_CONDITION' where only a handful aren't either 'normal' or 'unknown'.

'PEDPEDAL_ACTION' refers to what a pedestrian or cyclist was doing if they were involved in the accident. This would be useful if we were just looking at such accidents, but we are investigating all traffic crashes. Let's see the extent to which these ped/pedal accidents make up the total:

In [ ]:
pedpedal_action_vc = people_df['PEDPEDAL_ACTION'].value_counts()
pedpedal_action_vc_sum = sum(pedpedal_action_vc)
pedpedal_action_vc_sum / people_df.shape[0]

0.01903057395234521

It looks like this would only apply to less than 2% of the total rows, so it will only really add noise, so let's drop it along with 'PEDPEDAL_VISIBILITY' and 'PEDPEDAL_LOCATION'.

In [ ]:
people_df['BAC_RESULT'].value_counts()

BAC_RESULT
TEST NOT OFFERED                   1427625
TEST REFUSED                         15087
TEST PERFORMED, RESULTS UNKNOWN       3496
TEST TAKEN                            2576
Name: count, dtype: int64

'BAC_RESULT' would be helpful but only a tiny percentage involve any tests taken, so it will not help here.

In [ ]:
people_df['CELL_PHONE_USE'].value_counts()

CELL_PHONE_USE
Y    752
N    407
Name: count, dtype: int64

Unfortunately, 'CELL_PHONE_USE' doesn't show enough entries to be useful, either. With that, we have at least a good starting list of columns to drop, so let's drop them and move on to the last part of this dataset.

In [ ]:
people_columns_to_drop = ['PERSON_ID', 'VEHICLE_ID', 'CRASH_DATE', 'SEAT_NO', 'STATE', 'ZIPCODE', 'DRIVERS_LICENSE_STATE',
                   'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL',
                   'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION',
                   'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'BAC_RESULT',
                   'BAC_RESULT VALUE', 'CELL_PHONE_USE']

In [ ]:
people_df_refined = people_df.drop(people_columns_to_drop, axis=1)
people_df_refined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1818705 entries, 0 to 1818704
Data columns (total 6 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   PERSON_TYPE            object 
 1   CRASH_RECORD_ID        object 
 2   CITY                   object 
 3   SEX                    object 
 4   AGE                    float64
 5   DRIVERS_LICENSE_CLASS  object 
dtypes: float64(1), object(5)
memory usage: 83.3+ MB


## vehicle data

Finally, let's examine the third part of this dataset on vehicles:

In [ ]:
vehicles_df = pd.read_csv('/content/CPD_traffic_crashes-vehicles-20240501.csv')
vehicles_df.head(3)

<ipython-input-33-04261ed8d1c9>:1: DtypeWarning: Columns (20,39,40,41,43,47,48,49,52,54,57,58,60,70) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles_df = pd.read_csv('/content/CPD_traffic_crashes-vehicles-20240501.csv')


,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1717556,7b1763088507f77e0e552c009a6bf89a4d6330c7527706...,12/06/2023 03:24:00 PM,1,DRIVER,NaN,1634931.0,NaN,NISSAN,SENTRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1717574,2603ff5a88f0b9b54576934c5ed4e4a64e8278e005687b...,12/06/2023 04:00:00 PM,2,DRIVER,NaN,1634978.0,NaN,CHRYSLER,SEBRING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1717579,a52ef70e33d468b855b5be44e8638a564434dcf99c0edf...,12/06/2023 04:30:00 PM,1,DRIVER,NaN,1634948.0,NaN,SUBARU,OUTBACK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
vehicles_df.shape

(1690173, 71)

In [ ]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1690173 entries, 0 to 1690172
Data columns (total 71 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1690173 non-null  int64  
 1   CRASH_RECORD_ID           1690173 non-null  object 
 2   CRASH_DATE                1690173 non-null  object 
 3   UNIT_NO                   1690173 non-null  int64  
 4   UNIT_TYPE                 1688017 non-null  object 
 5   NUM_PASSENGERS            250022 non-null   float64
 6   VEHICLE_ID                1651514 non-null  float64
 7   CMRC_VEH_I                31452 non-null    object 
 8   MAKE                      1651509 non-null  object 
 9   MODEL                     1651364 non-null  object 
 10  LIC_PLATE_STATE           1500227 non-null  object 
 11  VEHICLE_YEAR              1386705 non-null  float64
 12  VEHICLE_DEFECT            1651514 non-null  object 
 13  VEHICLE_TYPE              1

The first column that doesn't look too redundant is 'UNIT_TYPE':

In [ ]:
vehicles_df['UNIT_TYPE'].value_counts()

UNIT_TYPE
DRIVER                 1415231
PARKED                  222855
PEDESTRIAN               21244
DRIVERLESS               13847
BICYCLE                  12794
NON-MOTOR VEHICLE         1482
NON-CONTACT VEHICLE        306
DISABLED VEHICLE           251
EQUESTRIAN                   7
Name: count, dtype: int64

Almost everything here is either a driver-driven vehicle or a parked one, so I'll probably drop this one because I think we may find a more useful designation once we get to 'VEHICLE_TYPE'. Anyway, I hope those seven horses in the 'EQUESTRIAN' class were okay.

The next one to consider would be 'MAKE', but it turns out to have *way* too many different values and the resulting sparsity from the one-hot encoded columns would just add a lot of noise to our model.

In [ ]:
len(vehicles_df['MAKE'].value_counts())

1335

I'm sure the 'MODEL' column would be even worse in this regard, so let's drop them both and search for primary causes elsewhere in the name of model interpretability.

In [ ]:
vehicles_df['VEHICLE_DEFECT'].value_counts()

VEHICLE_DEFECT
NONE                892456
UNKNOWN             741379
OTHER                 9406
BRAKES                5177
TIRES                  839
STEERING               748
WHEELS                 419
SUSPENSION             269
FUEL SYSTEM            233
ENGINE/MOTOR           210
WINDOWS                111
LIGHTS                 101
CARGO                   57
SIGNALS                 42
RESTRAINT SYSTEM        25
TRAILER COUPLING        23
EXHAUST                 19
Name: count, dtype: int64

With almost every class being 'none' or 'unknown', this column won't help.

In [ ]:
vehicles_df['VEHICLE_TYPE'].value_counts()

VEHICLE_TYPE
PASSENGER                                 1038710
SPORT UTILITY VEHICLE (SUV)                226692
UNKNOWN/NA                                 152665
VAN/MINI-VAN                                78278
PICKUP                                      53326
TRUCK - SINGLE UNIT                         31040
OTHER                                       19918
BUS OVER 15 PASS.                           17739
TRACTOR W/ SEMI-TRAILER                     15661
BUS UP TO 15 PASS.                           4659
MOTORCYCLE (OVER 150CC)                      3869
SINGLE UNIT TRUCK WITH TRAILER               2734
OTHER VEHICLE WITH TRAILER                   2253
TRACTOR W/O SEMI-TRAILER                     2069
AUTOCYCLE                                     669
MOPED OR MOTORIZED BICYCLE                    544
MOTOR DRIVEN CYCLE                            328
ALL-TERRAIN VEHICLE (ATV)                     182
FARM EQUIPMENT                                 84
3-WHEELED MOTORCYCLE (2 REAR WHEELS) 

With 60% of new-vehicle registrations in the US being SUVs in 2023 and this dataset only going back to 2015, I find it hard to believe that SUV's are outnumbered at almost 4-to-1 in this count. I'm getting worried about this dataset's viability.

For the sake of brevity, 'VEHICLE_USE' is all either 'personal' or 'unknown' or 'not in use', so it won't be of much help.

'TRAVEL_DIRECTION' may prove useful, but the cardinal directions (N,S,E,W) are all *much* more frequent than the intermediate ones, so we'll just keep those rows with cardinals.

'MANEUVER' seems to be a really useful column when it comes to the primary cause of an accident, so we'll keep it, but we'll drop the rows that are 'unknown', 'other', or occured fewer than 1,000 times.

In [ ]:
vehicles_df['MANEUVER'].value_counts()

MANEUVER
STRAIGHT AHEAD                        761499
PARKED                                227171
UNKNOWN/NA                            126724
SLOW/STOP IN TRAFFIC                  122016
TURNING LEFT                           98255
BACKING                                66781
TURNING RIGHT                          54715
PASSING/OVERTAKING                     40106
CHANGING LANES                         31673
OTHER                                  27722
ENTERING TRAFFIC LANE FROM PARKING     19410
MERGING                                11451
U-TURN                                  9411
STARTING IN TRAFFIC                     9341
LEAVING TRAFFIC LANE TO PARK            7925
AVOIDING VEHICLES/OBJECTS               6988
SKIDDING/CONTROL LOSS                   6296
ENTER FROM DRIVE/ALLEY                  6001
PARKED IN TRAFFIC LANE                  5091
SLOW/STOP - LEFT TURN                   3042
DRIVING WRONG WAY                       2452
NEGOTIATING A CURVE                     2077
S

Let's also examine 'FIRST_CONTACT_POINT':

In [ ]:
vehicles_df['FIRST_CONTACT_POINT'].value_counts()

FIRST_CONTACT_POINT
FRONT                 323891
REAR                  218038
UNKNOWN               157277
SIDE-LEFT             113074
SIDE-RIGHT            107560
FRONT-LEFT-CORNER     104177
FRONT-RIGHT-CORNER    102005
FRONT-LEFT             81524
FRONT-RIGHT            76903
REAR-LEFT              68088
REAR-LEFT-CORNER       46318
OTHER                  41017
REAR-RIGHT             36488
REAR-RIGHT-CORNER      33782
TOTAL (ALL AREAS)      27475
SIDE-LEFT-REAR         26325
SIDE-RIGHT-REAR        19947
SIDE-LEFT-FRONT        17038
SIDE-RIGHT-FRONT       14665
NONE                   13141
ROOF                   11958
UNDER CARRIAGE          5728
TOP                     1970
Name: count, dtype: int64

This would be useful, but it's a little redundant and lengthy when compared to the crashes_df column 'FIRST_CRASH_TYPE.' The remaining columns are all too filled with NaN's, so we now have a short list of columns to keep:



In [ ]:
vehicles_df_columns_to_keep = ['CRASH_RECORD_ID', 'TRAVEL_DIRECTION', 'MANEUVER']

In [ ]:
vehicles_df_refined = vehicles_df[vehicles_df_columns_to_keep]
vehicles_df_refined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1690173 entries, 0 to 1690172
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   CRASH_RECORD_ID   1690173 non-null  object
 1   TRAVEL_DIRECTION  1651514 non-null  object
 2   MANEUVER          1651514 non-null  object
dtypes: object(3)
memory usage: 38.7+ MB


## merging the three trimmed tables together

Now we can merge these more managable, concise DataFrames together and then begin removing the rows mentioned above.

In [ ]:
#First, we'll merge the crashes & people df's:
merged_crashes = pd.merge(left=crashes_df_refined,
                          right=people_df_refined,
                          left_on='CRASH_RECORD_ID',
                          right_on='CRASH_RECORD_ID')

#Now the newly merged df and the vehicles one:
merged_crashes = pd.merge(left=merged_crashes,
                          right=vehicles_df_refined,
                          left_on='CRASH_RECORD_ID',
                          right_on='CRASH_RECORD_ID')

merged_crashes.head()

,CRASH_RECORD_ID,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,PRIM_CONTRIBUTORY_CAUSE,STREET_DIRECTION,NUM_UNITS,CRASH_HOUR,...,CRASH_MONTH,LATITUDE,LONGITUDE,PERSON_TYPE,CITY,SEX,AGE,DRIVERS_LICENSE_CLASS,TRAVEL_DIRECTION,MANEUVER
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,15,CLEAR,DAYLIGHT,DRY,NO DEFECTS,FOLLOWING TOO CLOSELY,W,2,12,...,8,NaN,NaN,DRIVER,CHICAGO,M,45.0,D,W,STRAIGHT AHEAD
1,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,15,CLEAR,DAYLIGHT,DRY,NO DEFECTS,FOLLOWING TOO CLOSELY,W,2,12,...,8,NaN,NaN,DRIVER,CHICAGO,M,45.0,D,W,STRAIGHT AHEAD
2,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,15,CLEAR,DAYLIGHT,DRY,NO DEFECTS,FOLLOWING TOO CLOSELY,W,2,12,...,8,NaN,NaN,DRIVER,WEST CHICAGO,M,69.0,DM,W,STRAIGHT AHEAD
3,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,15,CLEAR,DAYLIGHT,DRY,NO DEFECTS,FOLLOWING TOO CLOSELY,W,2,12,...,8,NaN,NaN,DRIVER,WEST CHICAGO,M,69.0,DM,W,STRAIGHT AHEAD
4,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,FAILING TO REDUCE SPEED TO AVOID CRASH,S,4,14,...,7,41.85412,-87.665902,DRIVER,NaN,X,NaN,NaN,E,TURNING RIGHT


## reevaluating some columns since we still have too many

I can already tell we'll have way to many columns once I one-hot encode these. Let's go back and see what other columns we can drop.

In [ ]:
merged_crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3820905 entries, 0 to 3820904
Data columns (total 21 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   POSTED_SPEED_LIMIT       int64  
 2   WEATHER_CONDITION        object 
 3   LIGHTING_CONDITION       object 
 4   ROADWAY_SURFACE_COND     object 
 5   ROAD_DEFECT              object 
 6   PRIM_CONTRIBUTORY_CAUSE  object 
 7   STREET_DIRECTION         object 
 8   NUM_UNITS                int64  
 9   CRASH_HOUR               int64  
 10  CRASH_DAY_OF_WEEK        int64  
 11  CRASH_MONTH              int64  
 12  LATITUDE                 float64
 13  LONGITUDE                float64
 14  PERSON_TYPE              object 
 15  CITY                     object 
 16  SEX                      object 
 17  AGE                      float64
 18  DRIVERS_LICENSE_CLASS    object 
 19  TRAVEL_DIRECTION         object 
 20  MANEUVER                 object 
dtypes: float

This is a start, but there will still be too many columns once the categorical features are preprocessed. Let's refine these features further.

'STREET_DIRECTION' and 'TRAVEL_DIRECTION' seem redundant; let's take a closer look.

In [ ]:
merged_crashes['STREET_DIRECTION'].value_counts()

STREET_DIRECTION
W    1375481
S    1284668
N     900965
E     259777
Name: count, dtype: int64

In [ ]:
merged_crashes['TRAVEL_DIRECTION'].value_counts()

TRAVEL_DIRECTION
N          895016
S          876699
W          806022
E          790553
UNKNOWN    188516
SE          51430
NW          46334
SW          38910
NE          38131
Name: count, dtype: int64

It seems like travel direction is more pertinent than street direction, so we'll drop the streets. Next, let's investigate 'DRIVERS_LICENSE_CLASS' more closely.

In [ ]:
merged_crashes['DRIVERS_LICENSE_CLASS'].value_counts()

DRIVERS_LICENSE_CLASS
D     1631460
A       63593
C       50777
B       49603
DM      27444
       ...   
AZ          1
TA          1
C2          1
TR          1
LT          1
Name: count, Length: 281, dtype: int64

The distribution here is pretty skewed, the second most frequent class is less than 4% of the most frequent, so I don't think it will be much help. Let's check out 'ROAD_DEFECT' as well.

In [ ]:
merged_crashes['ROAD_DEFECT'].value_counts()

ROAD_DEFECT
NO DEFECTS           3106729
UNKNOWN               655370
OTHER                  18539
WORN SURFACE           15601
RUT, HOLES             15499
SHOULDER DEFECT         6612
DEBRIS ON ROADWAY       2555
Name: count, dtype: int64

Similar story here; after the less-than-useful classes of 'unknown' and 'other', the first specific one is 'worn surface', but it only amounts to 0.6% of the 'no defects' class, so this column will likely only hinder our model.

In [ ]:
merged_crashes['ROADWAY_SURFACE_COND'].value_counts()

ROADWAY_SURFACE_COND
DRY                2853372
WET                 538952
UNKNOWN             274956
SNOW OR SLUSH       119143
ICE                  24736
OTHER                 8807
SAND, MUD, DIRT        939
Name: count, dtype: int64

In [ ]:
final_columns_to_drop = ['CRASH_RECORD_ID', 'STREET_DIRECTION', 'DRIVERS_LICENSE_CLASS', 'ROAD_DEFECT']
merged_crashes = merged_crashes.drop(final_columns_to_drop, axis=1)
merged_crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3820905 entries, 0 to 3820904
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   POSTED_SPEED_LIMIT       int64  
 1   WEATHER_CONDITION        object 
 2   LIGHTING_CONDITION       object 
 3   ROADWAY_SURFACE_COND     object 
 4   PRIM_CONTRIBUTORY_CAUSE  object 
 5   NUM_UNITS                int64  
 6   CRASH_HOUR               int64  
 7   CRASH_DAY_OF_WEEK        int64  
 8   CRASH_MONTH              int64  
 9   LATITUDE                 float64
 10  LONGITUDE                float64
 11  PERSON_TYPE              object 
 12  CITY                     object 
 13  SEX                      object 
 14  AGE                      float64
 15  TRAVEL_DIRECTION         object 
 16  MANEUVER                 object 
dtypes: float64(3), int64(5), object(9)
memory usage: 495.6+ MB


## eliminating problematic rows

Before we begin preprocessing this merged DataFrame, we need to go through and eliminate the rows that contain 'unknown', 'other', or 'not reported' values as well as those that represent only a tiny fraction of the feature's value counts. This will help reduce the large amount of records we have and the amount of columns that will result after one-hot encoding.

In [ ]:
merged_crashes['POSTED_SPEED_LIMIT'].value_counts()

POSTED_SPEED_LIMIT
30    2914904
35     273495
25     214413
20     127854
15      99682
10      63520
40      44133
45      29551
0       28913
5       15887
55       4126
50       1232
3         977
9         400
39        374
99        291
60        252
1         125
24        122
2          84
65         82
32         75
33         62
34         60
36         40
11         40
26         35
6          26
7          22
14         18
70         15
31         14
29         11
22         10
12          9
23          8
8           8
38          8
4           7
44          6
18          6
63          4
62          2
49          1
46          1
Name: count, dtype: int64

Our first rows to drop are in the 'POSTED_SPEED_LIMIT' column. There are a lot of unusual speed limit values that aren't divisible by 5 (like 39, 24, 32). Sometimes these are done on purpose to catch the driver's attention, but at the end of the day, these will probably add more noise than anything else to our dataset.

In [ ]:
import numpy as np
realistic_speed_limits = list(np.arange(0,75,5))
merged_crashes = merged_crashes[merged_crashes['POSTED_SPEED_LIMIT'].isin(realistic_speed_limits)]
merged_crashes['POSTED_SPEED_LIMIT'].value_counts()

POSTED_SPEED_LIMIT
30    2914904
35     273495
25     214413
20     127854
15      99682
10      63520
40      44133
45      29551
0       28913
5       15887
55       4126
50       1232
60        252
65         82
70         15
Name: count, dtype: int64

In [ ]:
merged_crashes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3818059 entries, 0 to 3820904
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   POSTED_SPEED_LIMIT       int64  
 1   WEATHER_CONDITION        object 
 2   LIGHTING_CONDITION       object 
 3   ROADWAY_SURFACE_COND     object 
 4   PRIM_CONTRIBUTORY_CAUSE  object 
 5   NUM_UNITS                int64  
 6   CRASH_HOUR               int64  
 7   CRASH_DAY_OF_WEEK        int64  
 8   CRASH_MONTH              int64  
 9   LATITUDE                 float64
 10  LONGITUDE                float64
 11  PERSON_TYPE              object 
 12  CITY                     object 
 13  SEX                      object 
 14  AGE                      float64
 15  TRAVEL_DIRECTION         object 
 16  MANEUVER                 object 
dtypes: float64(3), int64(5), object(9)
memory usage: 524.3+ MB


In [ ]:
merged_crashes.shape

(3818059, 17)

We have a ton of rows so we can afford to drop those which aren't complete and contain values such as 'unknown', 'other', and 'not reported'. This will help trim our data for better, more complete predicting power and also help it run a little faster.

In [ ]:
unhelpful_strings = ['UNKNOWN', 'OTHER', 'NOT REPORTED']
#merged_crashes = merged_crashes[merged_crashes.apply(lambda x: not x.str.contains(unhelpful_strings).any(), axis=1)]
merged_crashes = merged_crashes[~merged_crashes.isin(unhelpful_strings).any(axis=1)]
merged_crashes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3285846 entries, 0 to 3820904
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   POSTED_SPEED_LIMIT       int64  
 1   WEATHER_CONDITION        object 
 2   LIGHTING_CONDITION       object 
 3   ROADWAY_SURFACE_COND     object 
 4   PRIM_CONTRIBUTORY_CAUSE  object 
 5   NUM_UNITS                int64  
 6   CRASH_HOUR               int64  
 7   CRASH_DAY_OF_WEEK        int64  
 8   CRASH_MONTH              int64  
 9   LATITUDE                 float64
 10  LONGITUDE                float64
 11  PERSON_TYPE              object 
 12  CITY                     object 
 13  SEX                      object 
 14  AGE                      float64
 15  TRAVEL_DIRECTION         object 
 16  MANEUVER                 object 
dtypes: float64(3), int64(5), object(9)
memory usage: 451.2+ MB


In [ ]:
filter = merged_crashes['ROADWAY_SURFACE_COND'].str.contains('SAND, MUD, DIRT')
merged_crashes = merged_crashes[~filter]
merged_crashes['ROADWAY_SURFACE_COND'].value_counts()

ROADWAY_SURFACE_COND
DRY              2652968
WET               502429
SNOW OR SLUSH     108194
ICE                21492
Name: count, dtype: int64

Now we'll narrow our rows to only show crashes that occured in the city of Chicago:

In [ ]:
merged_crashes = merged_crashes[merged_crashes['CITY'] == 'CHICAGO']
merged_crashes['CITY'].value_counts()

CITY
CHICAGO    1764661
Name: count, dtype: int64

Now that we know that worked and helped narrow down the rows to a more pertinent subset, we can drop this column since it only contains one entry:

In [ ]:
merged_crashes.drop('CITY', axis=1, inplace=True)

We also need to filter out all rows with the 'passenger' class in the 'PERSON_TYPE' column since it is far less likely that they were a contributing cause to the accident than a driver, pedestrian, or cyclist was. We'll also remove the 'non-motor vehicle' and 'non-contact vehcile' rows since they are so few and far between that including them as one-hot encoded columns will likely just hinder our model.

In [ ]:
unhelpful_strings = ['PASSENGER', 'NON-MOTOR VEHICLE', 'NON-CONTACT VEHCILE']
merged_crashes = merged_crashes[~merged_crashes['PERSON_TYPE'].str.contains('|'.join(unhelpful_strings))]
merged_crashes['PERSON_TYPE'].value_counts()

PERSON_TYPE
DRIVER                 1390889
PEDESTRIAN               33216
BICYCLE                  17921
NON-CONTACT VEHICLE        234
Name: count, dtype: int64

Next, we'll remove some strange negative numbers from the 'AGE' column:

In [ ]:
merged_crashes = merged_crashes[merged_crashes['AGE'] >= 0]
merged_crashes['AGE'].sort_values(ascending=True)

778000       0.0
2209640      0.0
2209641      0.0
3032594      0.0
633927       0.0
           ...  
1350514    109.0
1350515    109.0
998442     110.0
998443     110.0
2550030    110.0
Name: AGE, Length: 1360657, dtype: float64

Let's also drop the classes from 'MANUEVER' that total fewer than 1,000 since there are 27 classes here. We'll also get rid of NaN's that are described as "UNKNOWN/NA" here.

In [ ]:
#First, the classes with fewer than 1,000 entries
value_counts = merged_crashes['MANEUVER'].value_counts()
merged_crashes = merged_crashes[merged_crashes['MANEUVER'].isin(value_counts[value_counts >= 1000].index)]
#Now the specific 'unknown/na' class
merged_crashes = merged_crashes[~merged_crashes['MANEUVER'].str.contains('UNKNOWN/NA')]
merged_crashes['MANEUVER'].value_counts()

MANEUVER
STRAIGHT AHEAD                        756252
SLOW/STOP IN TRAFFIC                  126185
TURNING LEFT                          104928
PARKED                                 61085
TURNING RIGHT                          50413
BACKING                                42770
PASSING/OVERTAKING                     34536
CHANGING LANES                         28586
ENTERING TRAFFIC LANE FROM PARKING     18851
MERGING                                10796
STARTING IN TRAFFIC                    10027
U-TURN                                  9190
AVOIDING VEHICLES/OBJECTS               6267
ENTER FROM DRIVE/ALLEY                  6136
SKIDDING/CONTROL LOSS                   5386
LEAVING TRAFFIC LANE TO PARK            4990
SLOW/STOP - LEFT TURN                   3231
DRIVING WRONG WAY                       1981
PARKED IN TRAFFIC LANE                  1893
SLOW/STOP - RIGHT TURN                  1862
SLOW/STOP - LOAD/UNLOAD                 1556
NEGOTIATING A CURVE                     1430
N

Finally, let's check and see how the date & time data looks. The [data dictionary tells us](https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if/about_data) that Sunday=1, Monday=2, etc.

In [ ]:
merged_crashes['CRASH_DAY_OF_WEEK'].value_counts()

CRASH_DAY_OF_WEEK
6    214902
5    190044
3    188664
4    188298
7    183837
2    176733
1    145873
Name: count, dtype: int64

So, this columns appears as a numerical one, but it's really a categorical column that will need to be one-hot encoded. The same goes for 'CRASH_MONTH':

In [ ]:
merged_crashes['CRASH_MONTH'].value_counts()

CRASH_MONTH
10    120866
9     113592
12    110459
8     109767
11    107705
7     106067
5     105809
3     105151
6     104840
4     103798
1     101542
2      98755
Name: count, dtype: int64

However, we'll keep the hour of day as a numerical column [since it can serve as one](https://stats.stackexchange.com/a/245887/378499) and it would also help keep dimensionality down.

# preparing target column

The goal of this study is to explain the underlying causes in vehicle crashes, our target column will be 'PRIM_CONTIBUTORY_CAUSE'. The only obstacle here is the amount of classes this column contains:

In [ ]:
merged_crashes['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 360254
FAILING TO YIELD RIGHT-OF-WAY                                                       204123
FOLLOWING TOO CLOSELY                                                               172406
FAILING TO REDUCE SPEED TO AVOID CRASH                                               75524
IMPROPER OVERTAKING/PASSING                                                          68473
IMPROPER TURNING/NO SIGNAL                                                           53604
IMPROPER LANE USAGE                                                                  51056
IMPROPER BACKING                                                                     40939
DISREGARDING TRAFFIC SIGNALS                                                         39556
NOT APPLICABLE                                                                       38616
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                               

In [ ]:
len(merged_crashes['PRIM_CONTRIBUTORY_CAUSE'].value_counts())

40

While we can drop the 'UNABLE TO DETERMINE' and 'NOT APPLICABLE' classes, that would still leave 38 classes, which is not going to lead to any meaningful results.

My solution is to generalize these causes into two categories with the end goal for the stakeholder in mind, which is reducing traffic accidents. Specifically, I'll group these into those causes which show the driver was at fault and those which don't, like so:




In [ ]:
#First, we'll drop the unhelpful classes
unhelpful_strings = ['UNABLE TO DETERMINE', 'NOT APPLICABLE']
merged_crashes = merged_crashes[~merged_crashes['PRIM_CONTRIBUTORY_CAUSE'].str.contains('|'.join(unhelpful_strings))]
merged_crashes['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

PRIM_CONTRIBUTORY_CAUSE
FAILING TO YIELD RIGHT-OF-WAY                                                       204123
FOLLOWING TOO CLOSELY                                                               172406
FAILING TO REDUCE SPEED TO AVOID CRASH                                               75524
IMPROPER OVERTAKING/PASSING                                                          68473
IMPROPER TURNING/NO SIGNAL                                                           53604
IMPROPER LANE USAGE                                                                  51056
IMPROPER BACKING                                                                     40939
DISREGARDING TRAFFIC SIGNALS                                                         39556
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  38568
WEATHER                                                                              23196
DISREGARDING STOP SIGN                                            

In [ ]:
driver_at_fault = ['FAILING TO YIELD RIGHT-OF-WAY', 'FOLLOWING TOO CLOSELY', 'FAILING TO REDUCE SPEED TO AVOID CRASH',
                    'IMPROPER OVERTAKING/PASSING', 'IMPROPER TURNING/NO SIGNAL', 'IMPROPER LANE USAGE',
                    'IMPROPER BACKING', 'DISREGARDING TRAFFIC SIGNALS', 'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE',
                    'DISREGARDING STOP SIGN', 'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
                    'DISTRACTION - FROM INSIDE VEHICLE', 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)',
                    'DRIVING ON WRONG SIDE/WRONG WAY', 'DISREGARDING OTHER TRAFFIC SIGNS', 'EXCEEDING SAFE SPEED FOR CONDITIONS',
                    'EXCEEDING AUTHORIZED SPEED LIMIT', 'CELL PHONE USE OTHER THAN TEXTING', 'DISREGARDING ROAD MARKINGS',
                    'TURNING RIGHT ON RED', 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',
                    'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)', 'TEXTING', 'RELATED TO BUS STOP',
                    'DISREGARDING YIELD SIGN', 'PASSING STOPPED SCHOOL BUS', 'BICYCLE ADVANCING LEGALLY ON RED LIGHT',
                    'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT']

In [ ]:
driver_not_at_fault = ['WEATHER', 'EQUIPMENT - VEHICLE CONDITION', 'PHYSICAL CONDITION OF DRIVER',
                       'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)', 'DISTRACTION - FROM OUTSIDE VEHICLE',
                       'ROAD ENGINEERING/SURFACE/MARKING DEFECTS', 'ROAD CONSTRUCTION/MAINTENANCE',
                       'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST']

In [ ]:
#engineer new feature with condensed causes
merged_crashes['TARGET'] = merged_crashes['PRIM_CONTRIBUTORY_CAUSE'].apply(lambda x: 1 if x in driver_at_fault else 0)
merged_crashes['TARGET'].value_counts()

TARGET
1    825449
0     64032
Name: count, dtype: int64

In [ ]:
merged_crashes['TARGET'].value_counts(normalize=True)

TARGET
1    0.928012
0    0.071988
Name: proportion, dtype: float64

It looks like we will have to contend with some substantial - but not extreme - target class imbalance when fitting our models. Just to be sure, let's take a random sample of our merged_crashes DataFrame to make sure things look right:

In [ ]:
merged_crashes.sample(n=50, random_state=210)

,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,PRIM_CONTRIBUTORY_CAUSE,NUM_UNITS,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,PERSON_TYPE,SEX,AGE,TRAVEL_DIRECTION,MANEUVER,TARGET
2993012,30,CLEAR,DAYLIGHT,DRY,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,2,14,3,6,41.793420,-87.703698,DRIVER,F,25.0,W,STRAIGHT AHEAD,1
91531,30,CLEAR,DAYLIGHT,DRY,DISREGARDING STOP SIGN,2,13,6,11,41.743407,-87.585242,DRIVER,M,77.0,W,STRAIGHT AHEAD,1
1441037,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,FAILING TO YIELD RIGHT-OF-WAY,2,20,5,6,41.769155,-87.626813,DRIVER,M,20.0,NW,TURNING LEFT,1
2229200,30,CLEAR,DAYLIGHT,DRY,IMPROPER TURNING/NO SIGNAL,2,14,6,12,41.891942,-87.622816,DRIVER,M,46.0,N,U-TURN,1
796124,30,CLEAR,DAYLIGHT,DRY,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,2,16,6,6,41.764008,-87.722499,DRIVER,F,20.0,E,STRAIGHT AHEAD,1
94317,30,CLEAR,DAYLIGHT,DRY,"EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST",2,14,3,12,41.837077,-87.704966,DRIVER,M,60.0,S,STRAIGHT AHEAD,0
2318142,30,CLEAR,DAYLIGHT,WET,WEATHER,2,16,1,10,41.905940,-87.625269,DRIVER,M,33.0,S,STRAIGHT AHEAD,0
2945267,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,IMPROPER OVERTAKING/PASSING,2,17,5,12,41.896271,-87.648795,DRIVER,M,70.0,E,CHANGING LANES,1
125927,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,2,0,2,3,41.707710,-87.636930,DRIVER,F,22.0,S,LEAVING TRAFFIC LANE TO PARK,1
1671609,30,CLEAR,DAYLIGHT,DRY,DISREGARDING TRAFFIC SIGNALS,3,11,6,12,41.844168,-87.724401,DRIVER,F,43.0,N,STRAIGHT AHEAD,1


Now we can drop the original 'PRIM_CONTRIBUTORY_CAUSE' column to avoid the multicollinearity that would arise (plus the exploded categorical columns). We can also drop the 'CITY' column since every accident here will have the same value ('CHICAGO').

In [ ]:
merged_crashes.drop('PRIM_CONTRIBUTORY_CAUSE', axis=1, inplace=True)
merged_crashes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889481 entries, 0 to 3820858
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   POSTED_SPEED_LIMIT    889481 non-null  int64  
 1   WEATHER_CONDITION     889481 non-null  object 
 2   LIGHTING_CONDITION    889481 non-null  object 
 3   ROADWAY_SURFACE_COND  889481 non-null  object 
 4   NUM_UNITS             889481 non-null  int64  
 5   CRASH_HOUR            889481 non-null  int64  
 6   CRASH_DAY_OF_WEEK     889481 non-null  int64  
 7   CRASH_MONTH           889481 non-null  int64  
 8   LATITUDE              884745 non-null  float64
 9   LONGITUDE             884745 non-null  float64
 10  PERSON_TYPE           889481 non-null  object 
 11  SEX                   889427 non-null  object 
 12  AGE                   889481 non-null  float64
 13  TRAVEL_DIRECTION      889481 non-null  object 
 14  MANEUVER              889481 non-null  object 
 15  TARG

Now that we have our columns in order, we can also drop records will NaN's. They only make up 0.5% of our records.

In [ ]:
merged_crashes.dropna(axis=0, inplace=True)
merged_crashes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 884691 entries, 9 to 3820858
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   POSTED_SPEED_LIMIT    884691 non-null  int64  
 1   WEATHER_CONDITION     884691 non-null  object 
 2   LIGHTING_CONDITION    884691 non-null  object 
 3   ROADWAY_SURFACE_COND  884691 non-null  object 
 4   NUM_UNITS             884691 non-null  int64  
 5   CRASH_HOUR            884691 non-null  int64  
 6   CRASH_DAY_OF_WEEK     884691 non-null  int64  
 7   CRASH_MONTH           884691 non-null  int64  
 8   LATITUDE              884691 non-null  float64
 9   LONGITUDE             884691 non-null  float64
 10  PERSON_TYPE           884691 non-null  object 
 11  SEX                   884691 non-null  object 
 12  AGE                   884691 non-null  float64
 13  TRAVEL_DIRECTION      884691 non-null  object 
 14  MANEUVER              884691 non-null  object 
 15  TARG

## finalizing trimmed dataset for project use

Now that we have our columns in place, I'll download this file so I can upload to my Kaggle page where I'll download it for my project notebook [found here](https://colab.research.google.com/drive/1uUvI_7ytHNdIKEZjJs73YlK-Y0rZjljM).

In [ ]:
from google.colab import files
merged_crashes.to_csv('trimmed_Chicago_crashes_data.csv', index=False)
files.download('trimmed_Chicago_crashes_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>